# Topic Modelling using Latent Semantic Analysis

### Import Required Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_colwidth", 200)

### Load Dataset

We will use the 20 Newsgroup dataset from sklearn for this exercise.

In [2]:
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

# check number of documents
len(documents)

11314

In [3]:
# check document categories
dataset.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

The dataset has 11,314 text documents distributed across 20 different newsgroups.

### Data Preprocessing

To start with, we will try to clean our text data as much as possible. We will remove the punctuations, numbers, special characters all in one step using the regex replace("[^a-zA-Z#]", " ") which will replace everything, except alphabets, with a space. Then we will remove shorter words because usually they don't contain much of the information. Finally we will make all the text lowercase to nullify case sensitivity.

In [4]:
news_df = pd.DataFrame({'document':documents})

# removing everything except alphabets`
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z#]", " ")

# removing short words
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

# make all text lowercase
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

It is a good practice to remove the stop-words from the text data as they are mostly clutter and hardly add carry any information. Stop-words are terms like 'it', 'they', 'am', 'been', 'about', 'because', 'while', etc. To remove the stop-words from the documents text we will have to tokenize the text, i.e., splitting the string of text into individual tokens or words. We will stitch the tokens back together once we have removed the stop-words

In [5]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

# tokenization
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) 

# remove stop-words
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

# de-tokenization
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
    
news_df['clean_doc'] = detokenized_doc

## Document-Term Matrix

This is the first step towards topic modeling. We will use sklearn's TfidfVectorizer to create a document-term matrix with 1000 terms.

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
                             max_features= 1000, # keep top 1000 terms 
                             max_df = 0.5, 
                             smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])

X.shape # check shape of the document-term matrix

(11314, 1000)

We could have used all the terms to create this matrix but that would need quite a lot of computation. Hence, we have restricted the number of features to 1000.

## Topic Modeling

The next step is to represent each and every term and document as a vector. We will use the document-term matrix and decompose it into multiple matrices. We will use sklearn's TruncatedSVD to perform the task of matrix decomposition. Since the data comes from 20 different newsgroups, let's try to have 20 topics for our text data. The number of topics can be specified by using the n_components parameter.

In [7]:
from sklearn.decomposition import TruncatedSVD

# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)

svd_model.fit(X)

len(svd_model.components_)

20

The components of svd_model are our topics and we can access them using svdmodel.components. Finally let's print a few most important words in each of the 20 topics and see how our model has done.

In [11]:
terms = vectorizer.get_feature_names()

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:20]
    print("Topic "+str(i+1)+": ")
    topics = []
    for t in sorted_terms:
        topics.append(t[0])
    print(topics)
    print('\n')

Topic 1: 
['like', 'know', 'people', 'think', 'good', 'time', 'thanks', 'make', 'right', 'want', 'need', 'really', 'problem', 'used', 'work', 'windows', 'year', 'said', 'believe', 'going']


Topic 2: 
['thanks', 'windows', 'card', 'drive', 'mail', 'file', 'advance', 'files', 'software', 'program', 'help', 'video', 'email', 'looking', 'disk', 'using', 'info', 'problem', 'window', 'know']


Topic 3: 
['game', 'team', 'year', 'games', 'season', 'players', 'good', 'play', 'hockey', 'league', 'player', 'drive', 'best', 'teams', 'baseball', 'better', 'runs', 'sale', 'like', 'played']


Topic 4: 
['drive', 'scsi', 'disk', 'hard', 'card', 'drives', 'problem', 'controller', 'floppy', 'power', 'apple', 'speed', 'work', 'computer', 'video', 'problems', 'monitor', 'used', 'internal', 'cable']


Topic 5: 
['windows', 'file', 'window', 'files', 'program', 'using', 'problem', 'running', 'version', 'team', 'screen', 'game', 'application', 'server', 'think', 'directory', 'display', 'drivers', 'manager'